In [1]:
#include <chrono>
#include <ctime>
#include <cmath>
#include <algoviz/SVG.hpp>
#include "Object.hpp"
#include "ScrollingImage.hpp"
#include "Player.hpp"
#include "Obstacle.hpp"
#include "Trunk.hpp"
#include "DangerZone.hpp"
#include "Crow.hpp"
#include "ObstacleManager.hpp"
#include "Bread.hpp"

using namespace std;

AlgoViz::clear();

/******************
 * Anzeigeobjekte *
 ******************/
SVG canvas = SVG(400, 400, 19, 20, "Projekt");

//Hintergrund & Flussufer
Image paperImage = Image("sprites/linedPaper.png",canvas.getWidth() / 2, canvas.getHeight() / 2, canvas.getWidth(), canvas.getHeight(), &canvas);
ScrollingImage backgroundWater = ScrollingImage("sprites/water.png", 20, canvas.getHeight() * 0.9, &canvas);
ScrollingImage upperShore = ScrollingImage("sprites/upperShore.png", 0, canvas.getHeight() * 0.05, &canvas);
ScrollingImage lowerShore = ScrollingImage("sprites/lowerShore.png", 380, canvas.getHeight() * 0.05, &canvas);

//Zentrale Spielobjekte
Player player = Player(200, 200, &canvas);
DangerZone *leftBorder = new DangerZone(&canvas, nullptr);
ObstacleManager obstacleManager = ObstacleManager(&canvas, leftBorder);
Bread bread = Bread(leftBorder, &player, &canvas);

//User Interface
Text fps = Text("", 2, 20, &canvas);

//Außerhalb des Spielbereichs Schwarz anzeigen
Rect outsideBoundsCover = Rect(-500, -500, canvas.getWidth() + 1000, canvas.getHeight() + 1000, &canvas);
outsideBoundsCover.setStrokeWidth(1000);
outsideBoundsCover.setFill(0, 0, 0, 0);

/*******************
 * Spielverwaltung *
 *******************/
bool continueGame = true;
string button = "";
double deltaTime = 0.05;
double totalGameTime = 0;
double noObstacleTime = 0;

In [2]:
void updateGame(string button, double deltaTime) {
    //Objektupdates
    player.update(button, deltaTime);
    obstacleManager.updateAll(deltaTime);
    bread.update(deltaTime);
    backgroundWater.move(-90 * deltaTime);
    upperShore.move(-10 * deltaTime);
    lowerShore.move(-10 * deltaTime);
    
    //Zeitupdates
    noObstacleTime -= deltaTime;
    totalGameTime += deltaTime;
    
    //Hindernisse erscheinen lassen
    if(noObstacleTime <= 0 && rand() % 100 < (50.0 + 0.5 * totalGameTime) * deltaTime) {
        int randomEnemy = rand() % 100;
        if (randomEnemy < 70) {
            obstacleManager.addTrunk();
            noObstacleTime = fmax(0.5, 1.0 - 0.006 * totalGameTime);
        } else {
            obstacleManager.addCrow(&player, leftBorder);
            noObstacleTime = 0.2 - 0.002 * totalGameTime;
        }
    }
    
    /* 
     * Spiel beenden bei:
     * Gegnerkollision, Uferkollision, Verlassen des Bildschirms (rechts), Q als Input
     */
    if(obstacleManager.hasCollidingObject(&player) || player.getCyMin() < canvas.getHeight() * 0.05 || player.getCyMax() > canvas.getHeight() * 0.95 || player.getX() > canvas.getWidth() || button == "q") {
        continueGame = false;
    }
}

In [3]:
AlgoViz::sleep(500);
while (continueGame) {
    /*
     * Die Dauer des vorherigen Schleifendurchlaufs
     * wird als Referenz dafür verwendet, wieviel
     * sich Objekte bewegen sollen.
     * Dadurch bewegen sie sich pro Sekunde immer
     * ungefähr gleich schnell, unabhängig von der
     * Dauer der Schleifendurchläufe.
     */
    auto start = chrono::system_clock::now();
    button = canvas.lastKey();
    
    fps.setText(to_string((int)(1 / deltaTime)) + " FPS");
    
    updateGame(button, deltaTime);
    
    outsideBoundsCover.toFront();
    
    auto end = std::chrono::system_clock::now();
    //Dauer des Schleifendurchlaufs ausrechnen und speichern
    chrono::duration<double> elapsed_seconds = end-start;
    deltaTime = elapsed_seconds.count();
}
fps.setText("Total Time: " + to_string((int)(totalGameTime)) + " Seconds");